In [48]:
import pandas as pd
import spacy

import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.base import TransformerMixin
from sklearn.pipeline import Pipeline

In [49]:
script = pd.read_csv('Game_of_Thrones_Script_count_senti.csv')
script.head()

,Release Date,Season,Episode,Episode Title,Name,Sentence,Tokenize Words,Tokenize Words Alphanumeric Only,Sentence Word Count,Row,Family Name,Sentiment Scores,Polarity,Subjectivity
0,2011-04-17,Season 1,Episode 1,Winter is Coming,waymar royce,What do you expect? They're savages. One lot s...,"['What', 'do', 'you', 'expect', '?', 'They', ""...","['What', 'do', 'you', 'expect', 'They', 'savag...",25,0,other,"Sentiment(polarity=-0.125, subjectivity=0.375)",-0.125,0.375
1,2011-04-17,Season 1,Episode 1,Winter is Coming,will,I've never seen wildlings do a thing like this...,"['I', ""'ve"", 'never', 'seen', 'wildlings', 'do...","['I', 'never', 'seen', 'wildlings', 'do', 'a',...",21,1,other,"Sentiment(polarity=0.0, subjectivity=0.0)",0.000,0.000
2,2011-04-17,Season 1,Episode 1,Winter is Coming,waymar royce,How close did you get?,"['How', 'close', 'did', 'you', 'get', '?']","['How', 'close', 'did', 'you', 'get']",5,2,other,"Sentiment(polarity=0.0, subjectivity=0.0)",0.000,0.000
3,2011-04-17,Season 1,Episode 1,Winter is Coming,will,Close as any man would.,"['Close', 'as', 'any', 'man', 'would', '.']","['Close', 'as', 'any', 'man', 'would']",5,3,other,"Sentiment(polarity=0.0, subjectivity=0.0)",0.000,0.000
4,2011-04-17,Season 1,Episode 1,Winter is Coming,gared,We should head back to the wall.,"['We', 'should', 'head', 'back', 'to', 'the', ...","['We', 'should', 'head', 'back', 'to', 'the', ...",7,4,other,"Sentiment(polarity=0.0, subjectivity=0.0)",0.000,0.000


In [50]:
top_20 = script.groupby(['Name', 'Family Name'])[['Sentence Word Count']].sum().reset_index().sort_values(by=['Sentence Word Count']
                                                                                         , ascending=[0])

In [51]:
script_top5 = script[script["Name"].isin(top_20["Name"][:5])]

In [52]:
script_top_20 = script[script["Name"].isin(top_20["Name"][:20])]

In [53]:
script_top_20 = script_top_20[["Sentence", "Name"]]

In [54]:
script_top_20.columns = ["Sentence", "Label"]

In [55]:
script_top_20.head()

,Sentence,Label
15,Go on. Father's watching.,jon snow
16,And your mother.,jon snow
18,Thank you.,sansa stark
21,"Don't think too much, Bran.",jon snow
22,Relax your bow arm.,robb stark


In [56]:
script_top_20.to_csv('Game_of_Thrones_Script_top20.csv', encoding='utf-8', index=False)

In [57]:
script_top5["Name"].unique()

array(['jon snow', 'jaime lannister', 'cersei lannister',
       'tyrion lannister', 'daenerys targaryen'], dtype=object)

In [58]:
import string
from spacy.lang.en.stop_words import STOP_WORDS
from spacy.lang.en import English

# Create our list of punctuation marks
punctuations = string.punctuation

# Create our list of stopwords
nlp = spacy.load('en')
stop_words = spacy.lang.en.stop_words.STOP_WORDS

# Load English tokenizer, tagger, parser, NER and word vectors
parser = English()

# Creating our tokenizer function
def spacy_tokenizer(sentence):
    # Creating our token object, which is used to create documents with linguistic annotations.
    mytokens = parser(sentence)

    # Lemmatizing each token and converting each token into lowercase
    mytokens = [ word.lemma_.lower().strip() if word.lemma_ != "-PRON-" else word.lower_ for word in mytokens ]

    # Removing stop words
    mytokens = [ word for word in mytokens if word not in stop_words and word not in punctuations ]

    # return preprocessed list of tokens
    return mytokens

In [59]:
# Custom transformer using spaCy
class predictors(TransformerMixin):
    def transform(self, X, **transform_params):
        # Cleaning Text
        return [clean_text(text) for text in X]

    def fit(self, X, y=None, **fit_params):
        return self

    def get_params(self, deep=True):
        return {}

# Basic function to clean the text
def clean_text(text):
    # Removing spaces and converting text into lowercase
    return text.strip().lower()

In [60]:
bow_vector = CountVectorizer(tokenizer = spacy_tokenizer, ngram_range=(1,2))

In [61]:
tfidf_vector = TfidfVectorizer(tokenizer = spacy_tokenizer)

In [62]:
from sklearn.model_selection import train_test_split

X = script_top5['Sentence'] # the features we want to analyze
ylabels = script_top5['Name'] # the labels, or answers, we want to test against

X_train, X_test, y_train, y_test = train_test_split(X, ylabels, test_size=0.3)

In [63]:
# Logistic Regression Classifier
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression()


# Create pipeline using Bag of Words
pipe = Pipeline([("cleaner", predictors()),
                 ('vectorizer', bow_vector),
                 ('classifier', classifier)])

# model generation
pipe.fit(X_train,y_train)

/home/maxisses/TechnologyStuff/nlp/lib/python3.6/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


Pipeline(memory=None,
         steps=[('cleaner', <__main__.predictors object at 0x7f14a53296a0>),
                ('vectorizer',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 2), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 t...\b\\w\\w+\\b',
                                 tokenizer=<function spacy_tokenizer at 0x7f14ac43fd08>,
                                 vocabulary=None)),
                ('classifier',
                 LogisticRegression(C=1.0, class_weight=None, dual=False,
                                    fit_intercept=True, intercept_scaling=1,
             

In [64]:
from sklearn import metrics
# Predicting with a test dataset
predicted = pipe.predict(X_test)

# Model Accuracy
print("Logistic Regression Accuracy:",metrics.accuracy_score(y_test, predicted))


Logistic Regression Accuracy: 0.38857466063348417


In [65]:
pipe.classes_

array(['cersei lannister', 'daenerys targaryen', 'jaime lannister',
       'jon snow', 'tyrion lannister'], dtype=object)

In [87]:
pipe.predict_proba(["Yes, and the dwarf loves his whine and the women"])

array([[0.05815035, 0.24910986, 0.03068634, 0.34340225, 0.3186512 ]])

In [67]:
pipe.predict_proba(["The high sparrow annoys me."])

array([[0.67871721, 0.04113006, 0.15739864, 0.07543604, 0.04731806]])

In [68]:
pipe.predict_proba(["I own the dragons."])

array([[0.15503389, 0.53765287, 0.05786516, 0.08193406, 0.16751403]])

In [69]:
pipe.predict_proba(["I can't fight anymore because I lost my hand."])

array([[0.06396884, 0.24445445, 0.43925265, 0.18432098, 0.06800308]])

In [70]:
pipe.predict_proba(["My father is Eddard Stark."])

array([[0.23081516, 0.02145236, 0.12624394, 0.31208422, 0.30940431]])

In [71]:
pipe.predict_proba(["My father is Tywin Lannister."])

array([[0.4024775 , 0.03383039, 0.1140324 , 0.13279097, 0.31686874]])

In [72]:
pipe.predict_proba(["The north remembers."])

array([[0.23885739, 0.2838366 , 0.11847671, 0.20951673, 0.14931258]])

The last model was generic. In the next step I will try to tune the model a little and use Season 1 - 7 for train and cross validation. Then let's see how well this model performs on Season 8.

In [73]:
X = script_top5[script_top5["Season"] != "Season 8"]['Sentence'] # the features we want to analyze
ylabels = script_top5[script_top5["Season"] != "Season 8"]['Name'] # the labels, or answers, we want to test against

X_holdout = script_top5[script_top5["Season"] == "Season 8"]['Sentence']#
ylabels_holdout = script_top5[script_top5["Season"] == "Season 8"]['Name']

In [74]:
# Logistic Regression Classifier
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression()

hyperparams = {'classifier__C': [0.001, 0.01, 0.1, 1, 10, 100, 1000], "classifier__solver": ["newton-cg", "liblinear"]}

# Create pipeline using Bag of Words
pipe2 = Pipeline([("cleaner", predictors()),
                 ('vectorizer', bow_vector),
                 ('classifier', classifier)])


grid = GridSearchCV(pipe2, param_grid = hyperparams, cv=10)

# model generation
grid.fit(X,ylabels)

GridSearchCV(cv=10, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('cleaner',
                                        <__main__.predictors object at 0x7f14a53296d8>),
                                       ('vectorizer',
                                        CountVectorizer(analyzer='word',
                                                        binary=False,
                                                        decode_error='strict',
                                                        dtype=<class 'numpy.int64'>,
                                                        encoding='utf-8',
                                                        input='content',
                                                        lowercase=True,
                                                        max_df=1.0,
                                                        max_features=None,
                                                       

In [75]:
grid.best_params_

{'classifier__C': 0.1, 'classifier__solver': 'newton-cg'}

In [76]:
grid.best_score_

0.35566109070157287

In [77]:
best_model = grid.best_estimator_

In [78]:
from sklearn import metrics

# Predicting with my holdout dataset (Season 8)
predictions = best_model.predict(X_holdout)
print("Logistic Regression Accuracy:",metrics.accuracy_score(ylabels_holdout, predictions))

Logistic Regression Accuracy: 0.3434343434343434


In [91]:
best_model.classes_

array(['cersei lannister', 'daenerys targaryen', 'jaime lannister',
       'jon snow', 'tyrion lannister'], dtype=object)

In [89]:
best_model.predict_proba(["Yes, and the dwarf loves his whine and the women"])

array([[0.11291686, 0.13675034, 0.09031177, 0.1563509 , 0.50367012]])

In [86]:
best_model.predict_proba(["The love for my children is above everything"])

array([[0.26870395, 0.17181349, 0.16113636, 0.1465391 , 0.2518071 ]])

In [94]:
best_model.predict_proba(["I will break the chains and free millions of people."])

array([[0.09152649, 0.30541641, 0.13231777, 0.23478667, 0.23595265]])

In [90]:
best_model.predict_proba(["I won't be able to fight ever again..."])

array([[0.10488837, 0.18229382, 0.19773547, 0.30237345, 0.2127089 ]])

In [93]:
best_model.predict_proba(["My father is Tywin Lannister"])

array([[0.21094134, 0.08340677, 0.14699467, 0.14782952, 0.4108277 ]])